In [112]:
import mne
import pandas as pd

In [113]:
file_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_eeg.bdf'
eeg_data = mne.io.read_raw_bdf(file_path, preload=True)

channels_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_channels.tsv'
channels_info = pd.read_csv(channels_tsv_path, sep='\t')

events_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_events.tsv'
events_info = pd.read_csv(events_tsv_path, sep='\t')

Extracting EDF parameters from /Users/omaraguilarjr/GATech/Project-InnerSpeech/Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 696319  =      0.000 ...  1359.998 secs...
   onset  duration trial_type
0      0    3999.0        six
1   3999    4000.0        six
2   7999    4000.0       four
3  11999    4000.0        ten
4  15999    4000.0       four


In [114]:
# Sort data into their channels
available_channels = set(eeg_data.ch_names).intersection(set(channels_info['name']))
eeg_data.pick_channels(list(available_channels))
montage = mne.channels.make_standard_montage('biosemi64')
eeg_data.set_montage(montage, on_missing='ignore')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawEDF | sub-01_ses-EEG_task-inner_eeg.bdf, 60 x 696320 (1360.0 s), ~318.8 MB, data loaded>

In [115]:
# Function used to bin data
events_info['onset'] = events_info['onset'] / 1000
events_info['duration'] = events_info['duration'] / 1000

def bin_eeg_data(eeg_data, events_info):
    headers = ['label'] + eeg_data.ch_names
    binned_data = []

    for _, event in events_info.iterrows():
        onset = event['onset']
        duration = event['duration']
        label = event['trial_type']
        #print(f'Event: {_}')
        #print(f'Onset: {onset}, Duration: {duration}, Label: {label}')
        
        start_sample = int(onset * eeg_data.info['sfreq'])
        stop_sample = int((onset + duration) * eeg_data.info['sfreq'])

        #print(f'Start: {start_sample}, Stop: {stop_sample}\n')
        event_dict = {'label': label}
        for channel in eeg_data.info['ch_names']:
            event_data = eeg_data.get_data(picks=channel, start=start_sample, stop=stop_sample)
            event_dict[channel] = event_data[0].flatten().tolist()
        
        binned_data.append(event_dict)

    binned_df = pd.DataFrame(binned_data, columns=headers)
    return binned_df

In [117]:
binned_df = bin_eeg_data(eeg_data, events_info)

In [120]:
print(len(binned_df['Fpz'][1]))

2048


In [121]:
binned_df.to_csv('../../Data/Processed/sub01_binned.csv', index=False)